In [ ]:
import os
import gmsh
import numpy as np
import import_ipynb
from matplotlib.tri import Triangulation
import matplotlib.pyplot as plt

# Importando notebooks diretamente
try:
    from fem_pre_processing import create_domain
    from fem_processing import boundary_conditions
    from fem_pos_processing import graph_results
except ModuleNotFoundError as e:
    print(f"Modules were not found: {e}")
except ImportError as e:
    print(f"Error in import: {e}")

# `create_mesh_domain()`

In [ ]:
def create_mesh_domain(element_type, BOUNDARY, MATERIAL, h, auto_save=True, view_mesh=False):
    type, order = element_type
    a = 8e-2  
    b = 4e-2 
    vertices = [(0, 0, 0), (a, 0, 0), (a, b, 0), (0, b, 0)]

    # Define a new model
    gmsh.initialize()
    gmsh.model.add("rectangular_guide")

    # Define the points of the domain.
    point_tags = create_domain.add_points(vertices, h)

    # Criar linhas para formar as bordas do quadrado
    line_tags = create_domain.add_lines(point_tags)
    
    # Criar um loop de linha e uma superfície plana
    surface_tag = create_domain.add_surface(line_tags)

    # Definindo as curvas de contorno de Dirichlet (dim=1)
    gmsh.model.addPhysicalGroup(dim=1, tags=line_tags, tag=BOUNDARY[0]['tag'], name=BOUNDARY[0]['name'])

    # Adicionar grupos físicos para Dim=2 (superfícies)
    gmsh.model.addPhysicalGroup(dim=2, tags=[surface_tag], tag=MATERIAL[0]['tag'], name=MATERIAL[0]['name'])

    # Gerar a malha 2D
    gmsh.option.setNumber("Mesh.SaveAll", 1)

    # Define a ordem dos elementos
    gmsh.model.mesh.generate(2)
    gmsh.model.mesh.setOrder(order)

    # Visualizar a malha no ambiente Gmsh (opcional)
    if view_mesh:
        gmsh.fltk.run()
    
    if auto_save:
        os.makedirs("pre_processing/mesh", exist_ok=True)
        file_path = f"pre_processing/mesh/rectangular_guide_domain_{type}{order}.msh"
        gmsh.write(file_path)
        gmsh.finalize()

# `apply_physics()`

In [2]:
def apply_physics(FINITE_ELEMENT, mesh_data):
    """
    Adiciona uma nova chave 'source' a cada dicionário em conn_data.
    
    Parâmetros:
    - mesh_data: Dicionário contendo os dados da malha.
    - element_type: Tuple (tipo do elemento, ordem).
    
    Retorna:
    - mesh_data: O dicionário atualizado com a chave 'source' em cada elemento de conn_data.
    """
    # Desempacotar o tipo de elemento
    type, element_order = FINITE_ELEMENT
    
    # Dictionary with all nodes in the mesh
    cell_data = mesh_data['cell']
    nodes_data = mesh_data['nodes']

    for key, cell in mesh_data['cell'].items():
        # Calcular o valor de 'a' de acordo com a física do problema
        mur = cell['material']['relative_magnetic_permeability']
        er = cell['material']['relative_electric_permittivity']

        # Adicionar a nova chave 'material' ao dicionário da célula
        cell['stiffness_a_value'] = 1
        cell['mass_a_value'] = 1
        
        # Dictionary with boundary nodes
        pec_nodes = {idx: node for idx, node in enumerate(cell['conn'])
                        if nodes_data[node]['bc']['type'] == 'PEC'}  

        bgt_nodes = {idx: node for idx, node in enumerate(cell['conn'])
                         if nodes_data[node]['bc']['type'] == 'BGT'}
        
        # Verifica tipo de fronteira absorvente (ABC)
        if len(pec_nodes) > 1:
            cell_data[key]['abc'] = {'type': 'PEC', 'conn_idx': pec_nodes}
            
        elif len(bgt_nodes) > 1:
            cell_data[key]['abc'] = {'type': 'BGT', 'conn_idx': bgt_nodes}
            
        else:
            cell_data[key]['abc'] = {'type': None, 'conn_idx': None}

        # Adicionar a nova chave 'source' ao dicionário da célula
        cell['source'] = {'type': 'free_source', 'value': 0}  
    
    return mesh_data

# `waveguide_modes()`

In [ ]:
def waveguide_modes(m_max, n_max, mode_type):
    """
    Calcula os modos TE ou TM para um guia de onda retangular.

    Parâmetros:
    a (float): Dimensão transversal a (em metros).
    b (float): Dimensão transversal b (em metros).
    m_max (int): Número máximo de modos para m.
    n_max (int): Número máximo de modos para n.
    mode_type (str): Tipo de modo, 'TE' ou 'TM'.

    Retorna:
    dict: Dicionário com os modos e seus respectivos kc.
    """
    # Dimensões do guia de onda retangular
    a, b = 8e-2, 4e-2 

    if mode_type not in ['TE', 'TM']:
        raise ValueError("mode_type deve ser 'TE' ou 'TM'")

    modes = {}
    for m in range(m_max + 1):
        for n in range(n_max + 1):
            if mode_type == 'TE' and m == 0 and n == 0:
                continue  # Excluir o modo (0,0) para TE
            if mode_type == 'TM' and (m == 0 or n == 0):
                continue  # Excluir modos onde m=0 ou n=0 para TM

            kc = np.sqrt((m * np.pi / a)**2 + (n * np.pi / b)**2)
            modes[f"{mode_type}_{m}{n}"] = kc

    # Ordenar os autovalores de te_modes e listar os cinco menores
    sorted_te_modes = sorted(modes.items(), key=lambda x: x[1])
    five_smallest_modes = sorted_te_modes[:m_max]

    # Imprimir os cinco menores autovalores dos modos TE
    print(f"\nCinco menores autovalores analíticos dos modos {mode_type}:")
    for mode, kc in five_smallest_modes:
        print(f"{mode}: kc2 = {kc**2:.3f} rad2/m2")
    
    return five_smallest_modes

# `calculate_error()`

In [ ]:
def calculate_error(reference_modes, computed_modes):
    """
    Calcula o erro entre os valores de referência (modos analíticos) e os modos computados numericamente.

    Parâmetros:
    reference_modes (list): Lista de tuplas representando os modos analíticos, onde cada tupla contém o nome do modo e o valor de kc.
    computed_modes (list): Lista de valores computados numericamente para os menores kc^2.

    Retorna:
    dict: Um dicionário contendo os erros absolutos para cada modo correspondente.
    """
    # Extrair os valores de kc do dicionário de referência
    reference_values = [kc for _, kc in reference_modes]

    # Verificar se os tamanhos são compatíveis
    if len(reference_values) != len(computed_modes):
        raise ValueError("O número de modos de referência e os modos computados devem ser iguais.")

    # Calcular kc^2 para os valores de referência
    reference_kc_squared = [kc**2 for kc in reference_values]

    # Calcular os erros percentuais
    errors = [abs(ref - comp)/ref * 100 for ref, comp in zip(reference_kc_squared, computed_modes)]

    # Retornar os erros em um dicionário
    errors_dict = {name: error for (name, _), error in zip(reference_modes, errors)}

    for mode, error in errors_dict.items():
        print(f"Modo {mode}: Erro percentual = {error:.4f}")

# `potential_and_gradient()`

In [4]:
def potential_and_gradient(x, y):
    return 0, np.zeros((2,1))

# `get_modes_pattern()`

In [ ]:
def get_modes_pattern(mesh_data, eigenvalues, eigenvectors, modes):
    # Solução aproximada
    for i, eigenvalue in enumerate(eigenvalues):
        # Solução aproximada global
        u_h = boundary_conditions.global_potentials_solution(mesh_data, eigenvectors[:, i])

        # Extraindo as coordenadas globais dos nós (x, y) e a matriz de conectividade
        xg, yg, conn = graph_results.structured_data(mesh_data)

        # Para elementos P1 (3 nós por elemento)
        triangulation = Triangulation(xg, yg, conn)

        plt.figure(figsize=(8, 6))
        plt.tricontour(triangulation, list(u_h.values()), cmap='viridis')
        plt.title(rf'Solução Aproximada para Autovalor $\lambda_{i}$ = {eigenvalue:.3e}. Modo {modes[i][0]}')
        plt.xlabel(r'$x$')
        plt.ylabel(r'$y$')
        plt.tight_layout()

Conversão do arquivo Jupyter Notebook para um script Python: ``python -m nbconvert --to script name.ipynb``

Belo Horizonte, Brazil. 2024.  
Adilton Junio Ladeira Pereira - adt@ufmg.br  
&copy; All rights reserved.